In [159]:
import pandas as pd
import numpy as np

In [160]:
df=pd.read_csv("blinkit_dynamic_pricing_1000_samples.csv")
df.head()


,Product Name,Base Price,Competitor Price,Demand,Supply,Expiry Days,Optimized Price
0,Kinder Joy,114,114,1,0,16,112
1,Bru Coffee 200g,437,449,0,1,27,429
2,Bournvita 500g,125,126,2,1,18,123
3,Dairy Milk Chocolate,144,148,0,1,15,137
4,Amul Milk 1L,66,65,2,1,12,63


In [161]:
df.isnull().sum()

Product Name        0
Base Price          0
Competitor Price    0
Demand              0
Supply              0
Expiry Days         0
Optimized Price     0
dtype: int64

In [162]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import mean_absolute_error, r2_score

In [163]:
# encoder = LabelEncoder()
# df["Demand"] = encoder.fit_transform(df["Demand"])  # Convert 'High', 'Medium', 'Low' → 2,1,0
# df["Supply"] = encoder.fit_transform(df["Supply"])

In [164]:
df.head()

,Product Name,Base Price,Competitor Price,Demand,Supply,Expiry Days,Optimized Price
0,Kinder Joy,114,114,1,0,16,112
1,Bru Coffee 200g,437,449,0,1,27,429
2,Bournvita 500g,125,126,2,1,18,123
3,Dairy Milk Chocolate,144,148,0,1,15,137
4,Amul Milk 1L,66,65,2,1,12,63


In [165]:

from sklearn.ensemble import RandomForestRegressor

In [166]:
# Initialize the Random Forest Regressor
rf_model = RandomForestRegressor(n_estimators=200, random_state=42)

# Train the model
rf_model.fit(X_train, y_train)

# Predict on test data
y_pred = rf_model.predict(X_test)

In [167]:
r2_score(y_test,y_pred)*100

99.93175329054999

In [168]:
train_score = rf_model.score(X_train, y_train)  # R² score on training set
test_score = rf_model.score(X_test, y_test)  # R² score on test set

print(f"Training R² Score: {train_score:.4f}")
print(f"Testing R² Score: {test_score:.4f}")


Training R² Score: 0.9999
Testing R² Score: 0.9993


In [169]:
res=rf_model.predict([[125,126,2,1,18]])
res

d:\Anaconda\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(


array([485.295])

In [170]:
l=[]
for i in range(1,200):
    rf=RandomForestRegressor(n_estimators=i,random_state=42)
    rf.fit(X_train,y_train)
    y_pred=rf.predict(X_test)
    l.append(r2_score(y_test,y_pred)*100)



In [171]:
max(l)

99.93246344195582

In [5]:
# import pandas as pd
# import numpy as np
# import xgboost as xgb
# from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import LabelEncoder, StandardScaler
# from sklearn.metrics import mean_absolute_error, r2_score

# # Load dataset
# df = pd.read_csv("blinkit_dynamic_pricing_1000_samples.csv")

# # Clean price columns by removing the "₹" symbol and converting to float
# df["Base Price"] = df["Base Price"].replace({'₹': ''}, regex=True).astype(float)
# df["Competitor Price"] = df["Competitor Price"].replace({'₹': ''}, regex=True).astype(float)
# df["Optimized Price"] = df["Optimized Price"].replace({'₹': ''}, regex=True).astype(float)

# # Encode categorical variables (Demand, Supply)
# # encoder = LabelEncoder()
# # df["Demand"] = encoder.fit_transform(df["Demand"])   # e.g., High -> 2, Medium -> 1, Low -> 0
# # df["Supply"] = encoder.fit_transform(df["Supply"])

# # Define features (X) and target (y)
# X = df[["Base Price", "Competitor Price", "Demand", "Supply", "Expiry Days"]]
# y = df["Optimized Price"]

# # Normalize numerical features using StandardScaler
# scaler = StandardScaler()
# X[["Base Price", "Competitor Price", "Expiry Days"]] = scaler.fit_transform(X[["Base Price", "Competitor Price", "Expiry Days"]])

# # Split dataset into training and testing sets (80% train, 20% test)
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# # Initialize and train the XGBoost Regressor
# xgb_model = xgb.XGBRegressor(n_estimators=200, learning_rate=0.1, max_depth=6, random_state=42)
# xgb_model.fit(X_train, y_train)

# # Predict on the test set
# y_pred = xgb_model.predict(X_test)

# # Evaluate the model
# mae = mean_absolute_error(y_test, y_pred)
# r2 = r2_score(y_test, y_pred) * 100

# print(f"Mean Absolute Error (MAE): {mae:.2f}")
# print(f"R² Score: {r2:.4f}")

import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, r2_score
import pickle

# Load dataset
df = pd.read_csv("blinkit_dynamic_pricing_1000_samples.csv")

# Clean price columns by removing the "₹" symbol and converting to float
df["Base Price"] = df["Base Price"].replace({'₹': ''}, regex=True).astype(float)
df["Competitor Price"] = df["Competitor Price"].replace({'₹': ''}, regex=True).astype(float)
df["Optimized Price"] = df["Optimized Price"].replace({'₹': ''}, regex=True).astype(float)

# For this example, we assume Demand and Supply columns are already numeric.
# If they are categorical, you would need to encode them.

# Define features (X) and target (y)
X = df[["Base Price", "Competitor Price", "Demand", "Supply", "Expiry Days"]]
y = df["Optimized Price"]

# Normalize numerical features using StandardScaler
scaler = StandardScaler()
numerical_cols = ["Base Price", "Competitor Price", "Expiry Days"]
X.loc[:, numerical_cols] = scaler.fit_transform(X.loc[:, numerical_cols])

# Optionally, save the scaler for future predictions
with open("scaler.pkl", "wb") as f:
    pickle.dump(scaler, f)

# Split dataset into training and testing sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize and train the XGBoost Regressor
xgb_model = xgb.XGBRegressor(n_estimators=200, learning_rate=0.1, max_depth=6, random_state=42)
xgb_model.fit(X_train, y_train)

# Predict on the test set
y_pred = xgb_model.predict(X_test)

# Evaluate the model
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred) * 100

print(f"Mean Absolute Error (MAE): {mae:.2f}")
print(f"R² Score: {r2:.4f}")

# Save the trained model to a pickle file for later use
with open("xgb_model.pkl", "wb") as f:
    pickle.dump(xgb_model, f)

print("✅ Model and scaler have been saved!")


C:\Users\ssona\AppData\Local\Temp\ipykernel_2084\1203688443.py:72: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[-0.18602289  1.28504238  0.08144352 -0.3197561  -0.72095571  1.55250879
 -0.72095571 -1.25588854  1.15130917 -0.72095571  1.41877558  0.34890993
  1.01757597 -0.72095571 -0.72095571 -0.3197561   1.28504238  0.75010955
 -0.72095571  0.61637635 -1.12215533 -1.25588854  1.41877558 -1.12215533
  0.88384276  0.21517673 -1.12215533  1.01757597  0.34890993  1.41877558
 -0.58722251  1.686242   -0.58722251 -1.12215533  0.88384276 -0.85468892
  1.686242    0.75010955 -0.98842213 -1.65708816  1.41877558  0.21517673
  1.686242   -0.98842213  0.75010955  0.34890993  0.08144352  0.88384276
 -0.58722251 -0.4534893   1.41877558  0.88384276 -1.65708816 -1.25588854
 -0.05228968 -0.4534893   0.61637635  0.34890993  0.88384276 -1.25588854
  1.15130917 -1.38962175  1.686242   -1.65708816  1.15130917 -1.52335495
  0.21517673

Mean Absolute Error (MAE): 1.49
R² Score: 99.9761
✅ Model and scaler have been saved!


In [4]:
import pandas as pd
import numpy as np

# Example new input data (replace these values with your actual new sample)
new_sample = {
    "Base Price": [27],
    "Competitor Price": [30],
    "Demand": [1],      # Categorical input
    "Supply": [2],    # Categorical input
    "Expiry Days": [8]
}
new_df = pd.DataFrame(new_sample)

# Assume that 'encoder' and 'scaler' are already fitted on your training data.
# For example:
# encoder = LabelEncoder() and then fitted on the 'Demand' and 'Supply' columns in your training set.
scaler = StandardScaler()# and then fitted on the numeric columns.

# Encode categorical variables
# new_df["Demand"] = encoder.transform(new_df["Demand"])
# new_df["Supply"] = encoder.transform(new_df["Supply"])

# Scale numerical features
new_df[["Base Price", "Competitor Price", "Expiry Days"]] = scaler.transform(
    new_df[["Base Price", "Competitor Price", "Expiry Days"]]
)

# Predict using your trained XGBoost model (xgb_model)
predicted_price = xgb_model.predict(new_df)[0]
print(f"Predicted Optimized Price: {predicted_price:.2f}")


NameError: name 'StandardScaler' is not defined

In [177]:
import pickle

In [178]:
with open("xgb_model.pkl", "wb") as f:
    pickle.dump(xgb_model, f)

In [ ]:
import torch
import clip
from PIL import Image
import numpy as np

# Set device
device = "cuda" if torch.cuda.is_available() else "cpu"

# Load the CLIP model and its preprocessing function
model, preprocess = clip.load("ViT-B/32", device=device)

# List of product names (your catalog)
product_names = [
    "Amul Milk 1L", "Aashirvaad Atta 5kg", "Parle-G Biscuits", "Maggi Noodles (Pack of 2)", "Nestle Coffee 200g",
    "Bournvita 500g", "Haldiram Namkeen 250g", "Dairy Milk Chocolate", "Red Label Tea 250g", "Tata Salt 1kg",
    "Kissan Ketchup 500g", "Everest Masala 100g", "Sprite 1.25L", "Coca-Cola 1.25L", "Pepsi 1.25L",
    "Britannia Cheese Slices 200g", "Saffola Gold Oil 1L", "Fortune Basmati Rice 5kg", "Dettol Handwash 250ml",
    "Colgate Toothpaste 200g", "Nivea Body Lotion 400ml", "Vim Dishwash Liquid 500ml", "Lizol Floor Cleaner 500ml",
    "Surf Excel Matic 1kg", "Harpic Toilet Cleaner 500ml", "Sensodyne Toothpaste 100g", "Tide Detergent Powder 2kg",
    "Dove Shampoo 650ml", "Pantene Conditioner 300ml", "Himalaya Face Wash 100ml", "Lifebuoy Handwash 250ml",
    "Patanjali Aloe Vera Gel 150ml", "Horlicks Health Drink 500g", "Complan Nutrition 1kg", "Boost 500g",
    "Quaker Oats 1kg", "Kellogg’s Corn Flakes 1kg", "Sundrop Peanut Butter 500g", "Nutella 350g", "Mother Dairy Paneer 200g",
    "Britannia Rusk 300g", "Lay’s Potato Chips 90g", "Bingo Mad Angles 80g", "Sunfeast Dark Fantasy 75g",
    "Hide & Seek Fab Biscuits", "Good Day Butter Cookies", "Perk Chocolate", "5 Star Chocolate", "KitKat Chocolate",
    "Cadbury Silk", "Toblerone Chocolate", "Kinder Joy", "Kissan Mixed Fruit Jam 500g", "Sundrop Lite Oil 1L",
    "Fortune Mustard Oil 1L", "Tata Tea Gold 250g", "Bru Coffee 200g", "Borges Olive Oil 500ml", "Dabur Honey 500g",
    "Patanjali Cow Ghee 1L", "Gits Gulab Jamun Mix 200g", "MTR Instant Upma 200g", "Kellogg’s Muesli 500g",
    "Weikfield Baking Powder 100g", "Weikfield Custard Powder 100g", "Tata Sampann Dal 1kg", "Chana Dal 1kg",
    "Arhar Dal 1kg", "Rajma 1kg", "Kabuli Chana 1kg", "Moong Dal 1kg", "Whole Wheat Flour 5kg", "Gram Flour 1kg",
    "Sugar 1kg", "Rock Salt 1kg", "Cooking Soda 100g", "MTR Rasam Powder 100g", "Everest Garam Masala 100g",
    "MDH Chicken Masala 100g", "Everest Pav Bhaji Masala 100g", "Catch Chat Masala 100g", "Garam Masala 100g",
    "Schezwan Chutney 250g", "Tomato Ketchup 1kg", "Soy Sauce 500ml", "Coconut Oil 1L", "Groundnut Oil 1L"
]

# Compute text embeddings for all product names
text_inputs = clip.tokenize(product_names).to(device)
with torch.no_grad():
    text_features = model.encode_text(text_inputs)
# Normalize text features
text_features = text_features / text_features.norm(dim=-1, keepdim=True)

def predict_product_from_image(image_path):
    # Load and preprocess the image
    image = preprocess(Image.open(image_path)).unsqueeze(0).to(device)
    with torch.no_grad():
        image_features = model.encode_image(image)
    # Normalize image features
    image_features = image_features / image_features.norm(dim=-1, keepdim=True)
    
    # Compute cosine similarity between image and each product text embedding
    similarity = (100.0 * image_features @ text_features.T).cpu().numpy()
    best_idx = np.argmax(similarity)
    
    return product_names[best_idx], similarity[0][best_idx]

# Example usage: Provide a path to a sample product image
image_path = "Nest.jpeg"  # Replace with your actual image file
predicted_product, sim_score = predict_product_from_image(image_path)
print(f"Predicted Product: {predicted_product} (Similarity Score: {sim_score:.2f})")


Predicted Product: Lifebuoy Handwash 250ml (Similarity Score: 36.62)
